In [84]:
% matplotlib

Using matplotlib backend: TkAgg


In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
import csv

In [131]:
from nltk.stem import WordNetLemmatizer as WNL
lemmatizer = WNL()

In [219]:
from nltk import bigrams
from nltk import FreqDist

In [4]:
stop = stopwords.words('english')
stop.append('')
stop.append('none')

In [5]:
data = pd.read_csv('Data_All_190213\CSV\data.csv')

In [6]:
data.describe()

,Respondent ID,Collector ID,Email Address,First Name,Last Name,Custom Data 1
count,6.690000e+02,669.0,0.0,0.0,0.0,0.0
mean,6.935282e+09,170147291.0,NaN,NaN,NaN,NaN
std,4.761647e+07,0.0,NaN,NaN,NaN,NaN
min,6.782052e+09,170147291.0,NaN,NaN,NaN,NaN
25%,6.914592e+09,170147291.0,NaN,NaN,NaN,NaN
50%,6.955273e+09,170147291.0,NaN,NaN,NaN,NaN
75%,6.970708e+09,170147291.0,NaN,NaN,NaN,NaN
max,6.981979e+09,170147291.0,NaN,NaN,NaN,NaN


In [7]:
data.head()

,Respondent ID,Collector ID,Start Date,End Date,IP Address,Email Address,First Name,Last Name,Custom Data 1,What type of extensions are you interested in? (Check all that apply),...,What is your name?,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hand-tied Wefts,...,Full Name,Company,Address,Address 2,City/Town,State/Province,ZIP/Postal Code,Country,Email Address,Phone Number
1,6.981979e+09,170147291.0,02/12/2019 09:38:50 PM,02/12/2019 09:43:42 PM,24.117.41.58,NaN,NaN,NaN,NaN,Hand-tied Wefts,...,Michelle Kemper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.981956e+09,170147291.0,02/12/2019 06:11:59 PM,02/12/2019 06:15:19 PM,76.220.71.140,NaN,NaN,NaN,NaN,Hand-tied Wefts,...,Lindsey Moeller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.981947e+09,170147291.0,02/12/2019 05:05:32 PM,02/12/2019 05:07:23 PM,99.156.102.255,NaN,NaN,NaN,NaN,Hand-tied Wefts,...,Ashlyn Spencer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.981820e+09,170147291.0,02/12/2019 08:19:52 AM,02/12/2019 08:22:01 AM,107.77.211.168,NaN,NaN,NaN,NaN,Hand-tied Wefts,...,Amanda silvey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
data.columns

Index(['Respondent ID', 'Collector ID', 'Start Date', 'End Date', 'IP Address',
       'Email Address', 'First Name', 'Last Name', 'Custom Data 1', 'wefts?',
       'poly?', 'cylinder?', 'Are you a salon owner?',
       'How many years of extensions experience do you have?',
       'How urgent is your need for new extensions?',
       'What disappointments are you facing in your current situation or with your current supplier that caused you to want to make an account with us?',
       'What supplier are you currently struggling with? (This information is kept privately for internal purposes and will not be shared with the public).',
       'What's been holding you back from changing suppliers? What aspects of the relationship you have with your current supplier would you keep?',
       'Are you currently considering any other suppliers besides Hair & Compounds? If so, who? (Your answers are for internal research purposes only. Your answers will not be made public.)',
       'Can you d

In [9]:
idx = data.columns.get_loc("What is your name?")
data.drop(data.columns[idx+1:], axis = 1, inplace = True)

In [32]:
# drop columns before "wefts?"
idx = data.columns.get_loc("wefts?")
data.drop(data.columns[:idx], axis = 1, inplace = True)

In [33]:
# after removing the idx column indices
data.columns

Index(['wefts?', 'poly?', 'cylinder?', 'Are you a salon owner?',
       'How many years of extensions experience do you have?',
       'How urgent is your need for new extensions?',
       'What disappointments are you facing in your current situation or with your current supplier that caused you to want to make an account with us?',
       'What supplier are you currently struggling with? (This information is kept privately for internal purposes and will not be shared with the public).',
       'What's been holding you back from changing suppliers? What aspects of the relationship you have with your current supplier would you keep?',
       'Are you currently considering any other suppliers besides Hair & Compounds? If so, who? (Your answers are for internal research purposes only. Your answers will not be made public.)',
       'Can you describe your clientele to us? What are the main looks that your clients want to achieve? (Lengths, textures, fine hair, thick hair, curly hair, Barb

In [11]:
# restructure some column names

data.rename(columns = {data.filter(like = "What type of").columns.values[0]: 'wefts?', data.filter(like = "Unnamed").columns.values[0]: 'poly?', data.filter(like = "Unnamed").columns.values[1]: 'cylinder?'}, inplace= True)

In [34]:
data.columns

Index(['wefts?', 'poly?', 'cylinder?', 'Are you a salon owner?',
       'How many years of extensions experience do you have?',
       'How urgent is your need for new extensions?',
       'What disappointments are you facing in your current situation or with your current supplier that caused you to want to make an account with us?',
       'What supplier are you currently struggling with? (This information is kept privately for internal purposes and will not be shared with the public).',
       'What's been holding you back from changing suppliers? What aspects of the relationship you have with your current supplier would you keep?',
       'Are you currently considering any other suppliers besides Hair & Compounds? If so, who? (Your answers are for internal research purposes only. Your answers will not be made public.)',
       'Can you describe your clientele to us? What are the main looks that your clients want to achieve? (Lengths, textures, fine hair, thick hair, curly hair, Barb

In [36]:
data.iloc[0,:]

wefts?                                                                                                                                                                                                                                   Hand-tied Wefts
poly?                                                                                                                                                                                                                                   Polymer (Fusion)
cylinder?                                                                                                                                                                                                                                            NaN
Are you a salon owner?                                                                                                                                                                                                                               Yes
How 

In [38]:
# drop this first row which is kind of useless

#data.drop(data.index[[0]], inplace = True)

In [37]:
# finally the data
data.head()

,wefts?,poly?,cylinder?,Are you a salon owner?,How many years of extensions experience do you have?,How urgent is your need for new extensions?,What disappointments are you facing in your current situation or with your current supplier that caused you to want to make an account with us?,What supplier are you currently struggling with? (This information is kept privately for internal purposes and will not be shared with the public).,What's been holding you back from changing suppliers? What aspects of the relationship you have with your current supplier would you keep?,"Are you currently considering any other suppliers besides Hair & Compounds? If so, who? (Your answers are for internal research purposes only. Your answers will not be made public.)","Can you describe your clientele to us? What are the main looks that your clients want to achieve? (Lengths, textures, fine hair, thick hair, curly hair, Barbie-doll hair, hair loss clients, etc.)","If you could wave a magic wand and create the perfect relationship with your new supplier, what would that look like?",What is your name?
1,Hand-tied Wefts,Polymer (Fusion),NaN,Yes,15,I need to decide and place an order SOON. It's...,"Quality has gone down, price and out of stock",Bombshell and hair dreams,Fear of quality being good enough,Bellamy,"Length, thickness and hair loss","Quality, affordability, availability and grea...",Michelle Kemper
2,Hand-tied Wefts,NaN,NaN,No,3,I need to decide and place an order SOON. It's...,I don’t currently have a hand tied weft suppli...,NaN,N/a,Bohyme,Clients wanting length and/or fullness. Mostly...,Consistency with the quality of hair they are ...,Lindsey Moeller
3,Hand-tied Wefts,NaN,NaN,No,2,I need to decide and place an order SOON. It's...,colors constantly out of stock,Bombshell,Familiar with bombshell,No,Long tick looks,Colors in stock at good prices,Ashlyn Spencer
4,Hand-tied Wefts,NaN,NaN,Yes,7,I'm actively looking for a new supplier.,Always out of stock on colors,Bohyme,Nothing,No,Lengths and fullness,Just responsive,Amanda silvey
5,NaN,NaN,Cylinder (I-tip),No,2,I'm just passively browsing. No big deal. Just...,Unreliable rep and price,Shrink links,"Just knowledge of different lines, there are s...",No,"Shoulder to long length, thickness for thinnin...","Reliable, good communication, good quality hai...",Jacklyn Cabatic


In [307]:
struggles = data.filter(like = "What disappointment").dropna()
needs = data.filter(like= "If you could wave").dropna()
supplier = data.filter(like = "supplier are you").dropna()
clientele = data.filter(like = "Can you describe your clientele").dropna()
anxiety = data.filter(like= "What's been holding you back").dropna()

In [17]:
#data.to_csv("cleaned_SM.csv")

In [309]:
supplier.count()

What supplier are you currently struggling with? (This information is kept privately for internal purposes and will not be shared with the public).    547
dtype: int64

In [304]:
clientele.count()

Can you describe your clientele to us? What are the main looks that your clients want to achieve? (Lengths, textures, fine hair, thick hair, curly hair, Barbie-doll hair, hair loss clients, etc.)    651
dtype: int64

In [305]:
supplier.count()

Series([], dtype: int64)

In [261]:
def save_file(freq, fname, type = None):
    with open (fname, 'w', encoding = 'utf-8', newline = '') as out:
        csv_out = csv.writer(out)
       #csv_out.writerow(['word', 'frequency'])
        if type == "list":
            
            for i in range(0, len(freq)):
                csv_out.writerow([freq[i][0], freq[i][1]])
                
        else:
            
            for k, v in freq.items():
                csv_out.writerow([k, v])

### tokenize struggles

In [146]:
def m_tokenize(row):
    word_list = (word_tokenize(row.values.tolist()[0]))
    stop_removed_list = [re.sub('[^a-zA-Z0-9 -]', '', x) for x in word_list if (x.lower()) not in stop]
    stop_removed_list = [x.lower() for x in stop_removed_list if not x == '']
    
    return stop_removed_list

In [124]:
def make_tokens(df):
    tokens = df.apply(m_tokenize, axis= 1)
    return tokens

def token_frequency(tokens):
    all_words = list()
    frequency = dict()
    
    def all_to_list(row):
        for r in row:
            all_words.append(r)
            
    tokens.apply(all_to_list)
    
    for w in all_words:
        #w = w.lower()
        #w = re.sub('[\`\'\`\,\.\'\!\s\’\”\“\(\)\*\&]', '', w)
        
        if not w.strip():
            next
        elif w not in frequency:
            frequency[w] = 1
        elif w in frequency:
            frequency[w] += 1
    
    return frequency

### Lemmatization of tokens

In [184]:
def lemmatize_tokens(row):
    m_row = list()
    
    for r in row:
        m_row.append(lemmatizer.lemmatize(r))
    
    return m_row

### Bi-grams

In [283]:
def get_bigrams(row):
    m_row = list()
    
    if len(row) > 0:
    
        try:
            return(list(bigrams(row)))
        except:
            return None
    

In [22]:
## Another way to do it
#for i in map(list, struggles.values):
#    print(i)

In [23]:
#all_words = list()
#frequency = dict()
#def all_to_list(row):
#    for r in row:
#        all_words.append(r)

In [43]:
#struggles_tokens.apply(all_to_list)

In [44]:
#for w in all_words:
#    w = w.lower()
#    w = re.sub('[\`\'\`\,\.\'\!\s]', '', w)
#    if w not in frequency:
#        frequency[w] = 1
#    elif w in frequency:
#        frequency[w] += 1
        

### call functions

In [310]:
## make tokens
tokens_struggles = make_tokens(struggles)
tokens_anxiety = make_tokens(anxiety)
tokens_needs = make_tokens(needs)
tokens_clientele = make_tokens(clientele)
tokens_supplier = make_tokens(supplier)

In [311]:
## lemmatize tokens (Lexicon normalization --> convert words/tokens to their root form which is linguistically correct) 
tokens_struggles_lemma = tokens_struggles.apply(lemmatize_tokens)
tokens_anxiety_lemma = tokens_anxiety.apply(lemmatize_tokens)
tokens_needs_lemma = tokens_needs.apply(lemmatize_tokens)
tokens_clientele_lemma = tokens_clientele.apply(lemmatize_tokens)
tokens_supplier_lemma = tokens_supplier.apply(lemmatize_tokens)

In [312]:
bigrams_struggles_lemma = tokens_struggles_lemma.apply(get_bigrams)
bigrams_anxiety_lemma = tokens_anxiety_lemma.apply(get_bigrams)
bigrams_needs_lemma = tokens_needs_lemma.apply(get_bigrams)
bigrams_clientele_lemma = tokens_clientele_lemma.apply(get_bigrams)
bigrams_supplier_lemma = tokens_supplier_lemma.apply(get_bigrams)

In [313]:
## get the frequency of each token
frequency_struggles = token_frequency(tokens_struggles_lemma)
freq_anx = token_frequency(tokens_anxiety_lemma)
freq_needs = token_frequency(tokens_needs_lemma)
freq_client = token_frequency(tokens_clientele_lemma)
freq_supplier = token_frequency(tokens_supplier_lemma)

In [264]:
#bigrams_struggles_lemma.iloc[0]

In [315]:
save_file(frequency_struggles, "freq_struggles.csv")
save_file(freq_anx, "freq_anxieties.csv")
save_file(freq_needs, "freq_needs.csv")
save_file(freq_client, "freq_clientele.csv")
save_file(freq_supplier, "freq_supplier.csv")

### get frequency of bi-grams

In [297]:
def get_bigram_freq(bigrams_force_lemma):
    all_words = list()
    def all_to_list(row):
        try:
            for r in row:
                all_words.append(r)
        except:
            return None

    bigrams_force_lemma.apply(all_to_list)
        
    bigram_frequency_force = (FreqDist(all_words).items())
    return bigram_frequency_force

In [316]:
bigram_frequency_struggles = get_bigram_freq(bigrams_struggles_lemma)
bigram_freq_anx = get_bigram_freq(bigrams_anxiety_lemma)
bigram_freq_needs = get_bigram_freq(bigrams_needs_lemma)
bigram_freq_client = get_bigram_freq(bigrams_clientele_lemma)
bigram_freq_supplier = get_bigram_freq(bigrams_supplier_lemma)

In [317]:
save_file(list(bigram_frequency_struggles), "bigram_freq_struggles.csv", type = "list")
save_file(list(bigram_freq_anx), "bigram_freq_anxities.csv", type = "list")
save_file(list(bigram_freq_needs), "bigram_freq_needs.csv", type = "list")
save_file(list(bigram_freq_client), "bigram_freq_client.csv", type = "list")
save_file(list(bigram_freq_supplier), "bigram_freq_supplier.csv", type = "list")